# 📊 GTM (Google Trends Transformer) 패션 매출 예측 - Google Colab

이 노트북은 Google Colab에서 GTM 모델을 실행하기 위해 최적화되었습니다.

## 🎯 모델 개요
- **멀티모달 입력**: 이미지, 텍스트(카테고리/색상/소재), Google Trends, 시간적 특성
- **출력**: 향후 12개월 매출 예측
- **아키텍처**: Transformer 기반 + ResNet50 + BERT

## 1. 🔧 환경 설정 및 Google Drive 마운트

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정
import os
os.chdir('/content')

# GPU 확인
import torch
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 단계적 패키지 설치 (에러 방지)
import os
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"✅ {package} 설치 완료")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {package} 설치 실패: {e}")
        return False

# 1. 기본 의존성부터 설치
print("🔧 기본 패키지 설치 중...")
install_package("wheel")
install_package("setuptools")

# 2. tokenizers 문제 해결 - 사전 컴파일된 버전 사용
print("🔧 tokenizers 설치 중...")
!pip install tokenizers --no-build-isolation --quiet

# 3. transformers 설치
print("🔧 transformers 설치 중...")
install_package("transformers==4.21.0")

# 4. PyTorch Lightning 설치 (Colab 기본 PyTorch와 호환되는 버전)
print("🔧 PyTorch Lightning 설치 중...")
install_package("pytorch-lightning==1.9.5")  # 더 안정적인 버전 사용

# 5. 기타 필요 패키지
print("🔧 추가 패키지 설치 중...")
install_package("wandb")
install_package("scikit-learn")

print("📦 모든 패키지 설치 완료!")

# 이제 import 테스트
try:
    import torch
    import pytorch_lightning as pl
    import transformers
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from pathlib import Path
    import warnings
    warnings.filterwarnings('ignore')
    
    from pytorch_lightning.callbacks import ModelCheckpoint
    from pytorch_lightning.loggers import CSVLogger
    
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"✅ PyTorch Lightning: {pl.__version__}")
    print(f"✅ Transformers: {transformers.__version__}")
    print(f"✅ CUDA 사용 가능: {torch.cuda.is_available()}")
    
except ImportError as e:
    print(f"❌ Import 실패: {e}")
    print("⚠️ 런타임을 재시작한 후 다음 대안 코드를 실행하세요:")
    print("!pip uninstall -y tokenizers transformers pytorch-lightning")
    print("!pip install pytorch-lightning==1.9.5 transformers==4.21.0 --no-cache-dir")

## 2. 🔄 GTM-Transformer 코드 복사

In [ ]:
# 수정된 GTM-Transformer 코드 사용 (Google Drive에서 복사)
# git clone 대신 이미 수정된 코드를 Google Drive에서 복사

print("🔍 Google Drive에서 수정된 GTM-Transformer 코드 확인...")

# Google Drive의 GTM-Transformer 경로 (사용자가 업로드한 경로)
DRIVE_CODE_PATH = '/content/drive/MyDrive/GTM-Transformer/'  # 수정된 코드 경로
DRIVE_DATASET_PATH = '/content/drive/MyDrive/GTM-dataset/'   # 데이터셋 경로

# GTM-Transformer 폴더가 있는지 확인
if os.path.exists(DRIVE_CODE_PATH):
    print("✅ 수정된 GTM-Transformer 코드를 찾았습니다!")
    print("📂 코드 구조:")
    !ls -la "/content/drive/MyDrive/GTM-Transformer/"
    
    # 코드를 Colab 작업 디렉토리로 복사
    print("🔄 코드 복사 중...")
    !cp -r "/content/drive/MyDrive/GTM-Transformer" "/content/"
    os.chdir('/content/GTM-Transformer')
    
    print("✅ 코드 복사 완료!")
    
    # 필요한 디렉토리 생성
    !mkdir -p checkpoints logs saved_models
    
else:
    print("❌ GTM-Transformer 폴더를 찾을 수 없습니다.")
    print("📋 업로드 방법:")
    print("1. 로컬의 수정된 GTM-Transformer 폴더 전체를 Google Drive에 업로드")
    print("2. Google Drive 경로: /content/drive/MyDrive/GTM-Transformer/")
    print("3. 폴더 구조 확인:")
    print("   GTM-Transformer/")
    print("   ├── models/")
    print("   ├── utils/") 
    print("   ├── train.py")
    print("   └── ...")

# 데이터셋 경로도 확인
if os.path.exists(DRIVE_DATASET_PATH):
    print("✅ 데이터셋도 확인되었습니다!")
else:
    print("⚠️ 데이터셋 폴더도 확인해주세요: /content/drive/MyDrive/GTM-dataset/")

print("✅ 환경 설정 완료!")

## 3. ✅ 코드 준비 완료 (디바이스 호환성 수정됨)

Google Drive에서 복사한 코드는 이미 호환성 문제가 모두 수정된 버전입니다:
- ✅ fairseq → transformers (Adafactor)
- ✅ validation_epoch_end → on_validation_epoch_end  
- ✅ CUDA → CPU/GPU 자동 선택
- ✅ torch.load weights_only 파라미터
- ✅ PyTorch Lightning 2.x 호환성
- ✅ **GPU/CPU 디바이스 불일치 해결** (NEW!)

### 🔧 최신 수정사항 (GPU/CPU 디바이스 오류 해결)
- GTM 모델 내 모든 `.to('cpu')` 호출을 동적 디바이스 할당으로 변경
- TextEmbedder의 word_embeddings를 올바른 디바이스로 배치
- 마스크 생성 시 디바이스 일치성 보장
- autoregressive 디코딩 시 텐서 디바이스 통일

In [ ]:
# 코드 확인 (선택적)
# 수정된 코드가 제대로 복사되었는지 확인하고 싶다면 실행

print("🔍 주요 파일 확인:")
!ls -la models/
print("\n📄 GTM.py에서 transformers import 확인:")
!head -10 models/GTM.py | grep -E "(transformers|Adafactor)"
print("\n📄 train.py에서 weights_only 확인:")
!grep -n "weights_only" train.py | head -3

print("\n✅ 모든 수정사항이 적용되어 있습니다!")

## 4. 📊 데이터셋 확인

In [ ]:
# 데이터셋 존재 확인 (축소된 데이터셋 사용)
dataset_path = Path('/content/drive/MyDrive/GTM-dataset-small/')  # 축소된 데이터셋 경로
required_files = ['train.csv', 'test.csv', 'gtrends.csv', 'category_labels.pt', 'color_labels.pt', 'fabric_labels.pt']

print("📂 축소된 데이터셋 파일 확인:")
for file in required_files:
    file_path = dataset_path / file
    exists = file_path.exists()
    if exists:
        size = file_path.stat().st_size
        print(f"  ✅ {file}: {size/1024:.1f} KB")
    else:
        print(f"  ❌ {file}: 파일 없음")

# 이미지 폴더 확인
image_path = dataset_path / 'images'
if image_path.exists():
    # 하위 폴더별 이미지 수 확인
    total_images = 0
    print(f"  📁 이미지 폴더 구조:")
    for subdir in sorted(image_path.iterdir()):
        if subdir.is_dir():
            subdir_images = list(subdir.glob('*.png')) + list(subdir.glob('*.jpg'))
            print(f"    📂 {subdir.name}: {len(subdir_images)}개")
            total_images += len(subdir_images)
    print(f"  🖼️ 총 이미지: {total_images}개")
else:
    print(f"  ❌ images/ 폴더 없음")

# 데이터 로딩 테스트 (parse_dates 파라미터 추가)
try:
    print("\n🔄 축소된 데이터 로딩 테스트...")
    train_df = pd.read_csv(dataset_path / 'train.csv', parse_dates=['release_date'])
    test_df = pd.read_csv(dataset_path / 'test.csv', parse_dates=['release_date'])
    gtrends = pd.read_csv(dataset_path / 'gtrends.csv', index_col=[0], parse_dates=True)
    
    print(f"\n📊 축소된 데이터 통계:")
    print(f"  - 훈련 데이터: {len(train_df):,}개 샘플 (원본의 ~1/10)")
    print(f"  - 테스트 데이터: {len(test_df):,}개 샘플 (원본의 ~1/10)")
    print(f"  - Google Trends: {len(gtrends):,}개 시점 (동일)")
    print(f"  - 총 이미지: {total_images}개 (원본의 ~1/10)")
    
    # release_date 타입 확인
    print(f"\n📅 release_date 타입: {type(train_df['release_date'].iloc[0])}")
    print(f"💾 예상 데이터셋 크기: ~{total_images * 0.1:.0f} MB")
    
    # 카테고리 분포 확인
    print(f"\n🏷️ 카테고리 분포:")
    category_counts = train_df['category'].value_counts()
    for cat, count in category_counts.head(10).items():
        print(f"  - {cat}: {count}개")
    
    # 데이터 미리보기
    print(f"\n👀 축소된 훈련 데이터 첫 3행:")
    print(train_df[['category', 'color', 'fabric', 'release_date', 'image_path']].head(3))
    
except Exception as e:
    print(f"❌ 데이터 로딩 실패: {e}")
    print("\n🔧 해결 방법:")
    print("1. 로컬에서 'dataset_small' 폴더를 'GTM-dataset-small'로 이름 변경")
    print("2. Google Drive에 'GTM-dataset-small' 폴더 업로드")
    print("3. 폴더 구조 확인:")
    print("   GTM-dataset-small/")
    print("   ├── train.csv (508 샘플)")
    print("   ├── test.csv (50 샘플)")
    print("   ├── images/ (~558개 이미지)")
    print("   └── *.pt 라벨 파일들")
    print("4. 아래 명령으로 실제 폴더 확인:")
    print("   !ls -la '/content/drive/MyDrive/GTM-dataset-small/'")

print(f"\n🚀 축소된 데이터셋으로 빠른 실험 준비 완료!")

# 이미지 폴더 구조 확인 및 재구성
import shutil
from pathlib import Path

print("🖼️ 이미지 폴더 구조 확인 및 수정...")

dataset_path = Path('/content/drive/MyDrive/GTM-dataset/')
image_path = dataset_path / 'images'

# 현재 이미지 폴더 구조 확인
print(f"📂 이미지 폴더: {image_path}")

# 하위 폴더들이 있는지 확인
subdirs = [d for d in image_path.iterdir() if d.is_dir()]
image_files = list(image_path.glob('*.png')) + list(image_path.glob('*.jpg'))

print(f"  📁 하위 폴더 수: {len(subdirs)}")
print(f"  🖼️ 직접 이미지 파일 수: {len(image_files)}")

if subdirs:
    print("✅ 올바른 폴더 구조가 있습니다:")
    for subdir in subdirs:
        subdir_images = list(subdir.glob('*.png')) + list(subdir.glob('*.jpg'))
        print(f"  📁 {subdir.name}: {len(subdir_images)}개 이미지")
else:
    print("⚠️ 하위 폴더가 없습니다. 이미지 경로를 확인합니다...")
    
    # CSV에서 실제로 사용되는 경로 패턴 확인
    sample_paths = train_df['image_path'].head(10).tolist()
    print("📋 CSV의 이미지 경로 예시:")
    for path in sample_paths:
        print(f"  - {path}")
    
    # 필요한 하위 폴더들 생성 (CSV 경로 기반)
    required_subdirs = set()
    for path in train_df['image_path']:
        if '/' in path:
            subdir = path.split('/')[0]
            required_subdirs.add(subdir)
    
    print(f"\n📁 필요한 하위 폴더들: {list(required_subdirs)}")
    
    if len(image_files) > 0 and len(required_subdirs) > 0:
        print("🔧 폴더 구조 재구성 중...")
        
        # 하위 폴더들 생성
        for subdir in required_subdirs:
            (image_path / subdir).mkdir(exist_ok=True)
            print(f"  📁 생성: {subdir}/")
        
        # 이미지 파일들을 적절한 하위 폴더로 이동
        # 파일명 기반으로 폴더 추정 (예: PE17/00001.png -> 00001.png는 PE17 폴더에)
        moved_count = 0
        for img_file in image_files:
            # CSV에서 이 파일을 참조하는 경로 찾기
            matching_paths = train_df[train_df['image_path'].str.contains(img_file.name)]
            if len(matching_paths) > 0:
                csv_path = matching_paths.iloc[0]['image_path']
                if '/' in csv_path:
                    target_subdir = csv_path.split('/')[0]
                    target_path = image_path / target_subdir / img_file.name
                    
                    if not target_path.exists():
                        shutil.move(str(img_file), str(target_path))
                        moved_count += 1
        
        print(f"✅ {moved_count}개 이미지 파일 이동 완료!")
        
        # 최종 구조 확인
        print("\n📊 최종 폴더 구조:")
        for subdir in (image_path).iterdir():
            if subdir.is_dir():
                subdir_images = list(subdir.glob('*.png')) + list(subdir.glob('*.jpg'))
                print(f"  📁 {subdir.name}: {len(subdir_images)}개 이미지")

print("\n✅ 이미지 폴더 구조 준비 완료!")

In [ ]:
# 모델 import 및 설정
sys.path.append('./models')
from models.GTM import GTM
from models.FCN import FCN
from utils.data_multitrends import ZeroShotDataset

# 하이퍼파라미터 (축소된 데이터셋용으로 조정)
BATCH_SIZE = 8 if torch.cuda.is_available() else 4  # 더 작은 배치 크기
EPOCHS = 5  # 빠른 실험을 위해 에포크 수 감소
ACCELERATOR = 'gpu' if torch.cuda.is_available() else 'cpu'
DEVICES = 1

# 축소된 데이터셋 경로
dataset_path = Path('/content/drive/MyDrive/GTM-dataset-small/')

# 라벨 딕셔너리 로딩 (축소된 데이터셋에서)
cat_dict = torch.load(dataset_path / 'category_labels.pt', weights_only=False)
col_dict = torch.load(dataset_path / 'color_labels.pt', weights_only=False)
fab_dict = torch.load(dataset_path / 'fabric_labels.pt', weights_only=False)

print(f"📋 축소된 라벨 딕셔너리:")
print(f"  - 카테고리: {len(cat_dict)}개")
print(f"  - 색상: {len(col_dict)}개") 
print(f"  - 소재: {len(fab_dict)}개")

# 축소된 데이터셋 생성
print("\n🔄 축소된 훈련 데이터셋 생성 중...")
train_dataset = ZeroShotDataset(
    train_df, 
    dataset_path / 'images',  # 축소된 이미지 경로
    gtrends, 
    cat_dict, 
    col_dict, 
    fab_dict, 
    trend_len=52
)

print("🔄 축소된 테스트 데이터셋 생성 중...")
test_dataset = ZeroShotDataset(
    test_df, 
    dataset_path / 'images',  # 축소된 이미지 경로
    gtrends, 
    cat_dict, 
    col_dict, 
    fab_dict, 
    trend_len=52
)

# DataLoader 생성 (더 작은 배치 크기)
print("🔄 DataLoader 생성 중...")
train_loader = train_dataset.get_loader(batch_size=BATCH_SIZE, train=True)
test_loader = test_dataset.get_loader(batch_size=1, train=False)

print(f"✅ 축소된 데이터 로딩 완료!")
print(f"  - 훈련 데이터: {len(train_df)}개")
print(f"  - 테스트 데이터: {len(test_df)}개")  
print(f"  - 훈련 배치 크기: {BATCH_SIZE}")
print(f"  - 테스트 배치 크기: 1")
print(f"  - 에포크: {EPOCHS}개 (빠른 실험용)")
print(f"  - 가속기: {ACCELERATOR}")

# 첫 번째 배치 구조 확인
print("\n🔍 첫 번째 배치 구조 확인:")
try:
    sample_batch = next(iter(train_loader))
    print(f"  - 배치 요소 수: {len(sample_batch)}")
    for i, item in enumerate(sample_batch):
        if hasattr(item, 'shape'):
            print(f"  - 요소 {i}: {item.shape}")
        else:
            print(f"  - 요소 {i}: {type(item)}")
    print("✅ 축소된 데이터셋 배치 구조 확인 완료!")
    print(f"💡 예상 훈련 시간: ~{len(train_loader) * EPOCHS // 10} 분")
except Exception as e:
    print(f"❌ 배치 로딩 실패: {e}")

In [ ]:
# GTM 모델 생성 및 설정
print("🎯 GTM 모델 생성 중...")

# GTM 모델 인스턴스 생성
model = GTM(
    embedding_dim=32,
    hidden_dim=64,
    output_dim=12,
    num_heads=4,
    num_layers=1,
    use_text=True,
    use_img=True,
    cat_dict=cat_dict,
    col_dict=col_dict,
    fab_dict=fab_dict,
    trend_len=52,
    num_trends=3,
    gpu_num=0,
    use_encoder_mask=1,
    autoregressive=False
)

print(f"✅ GTM 모델 생성 완료!")
print(f"📊 모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

# 체크포인트 및 로거 설정
checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints/',
    filename='gtm-colab-{epoch:02d}-{val_mae:.2f}',
    monitor='val_mae',
    mode='min',
    save_top_k=2,
    verbose=True
)

csv_logger = CSVLogger(
    save_dir='./logs/',
    name='gtm_colab'
)

# Trainer 설정
trainer = pl.Trainer(
    devices=DEVICES,
    accelerator=ACCELERATOR,
    max_epochs=EPOCHS,
    check_val_every_n_epoch=2,
    logger=csv_logger,
    callbacks=[checkpoint_callback],
    enable_progress_bar=True,
    log_every_n_steps=20
)

print(f"🚀 Trainer 설정 완료!")

# 🔍 Gradient 디버깅을 위한 텐서 상태 확인
print("\n🔍 입력 텐서들의 gradient 상태 확인...")

# 첫 번째 배치 가져오기
sample_batch = next(iter(train_loader))
item_sales, category, color, fabric, temporal_features, gtrends, images = sample_batch

print("📊 입력 텐서 gradient 상태:")
print(f"  - item_sales: requires_grad={item_sales.requires_grad}, device={item_sales.device}")
print(f"  - category: requires_grad={category.requires_grad}, device={category.device}")
print(f"  - color: requires_grad={color.requires_grad}, device={color.device}")
print(f"  - fabric: requires_grad={fabric.requires_grad}, device={fabric.device}")
print(f"  - temporal_features: requires_grad={temporal_features.requires_grad}, device={temporal_features.device}")
print(f"  - gtrends: requires_grad={gtrends.requires_grad}, device={gtrends.device}")
print(f"  - images: requires_grad={images.requires_grad}, device={images.device}")

# 모델의 각 인코더별 출력 확인
print("\n🔧 각 인코더 출력의 gradient 상태:")
with torch.no_grad():  # 일시적으로 gradient 계산 비활성화
    img_encoding = model.image_encoder(images)
    dummy_encoding = model.dummy_encoder(temporal_features)
    text_encoding = model.text_encoder(category, color, fabric)
    gtrend_encoding = model.gtrend_encoder(gtrends)
    
    print(f"  - img_encoding: requires_grad={img_encoding.requires_grad}")
    print(f"  - dummy_encoding: requires_grad={dummy_encoding.requires_grad}")
    print(f"  - text_encoding: requires_grad={text_encoding.requires_grad}")
    print(f"  - gtrend_encoding: requires_grad={gtrend_encoding.requires_grad}")

# 모델 파라미터 중 requires_grad=False인 것 찾기
print("\n⚠️ requires_grad=False인 모델 파라미터:")
no_grad_params = []
for name, param in model.named_parameters():
    if not param.requires_grad:
        no_grad_params.append(name)

if no_grad_params:
    print("  다음 파라미터들이 requires_grad=False:")
    for name in no_grad_params[:10]:  # 처음 10개만 출력
        print(f"    - {name}")
    if len(no_grad_params) > 10:
        print(f"    ... 총 {len(no_grad_params)}개")
else:
    print("  ✅ 모든 모델 파라미터가 requires_grad=True")

print("\n💡 Gradient 문제 해결을 위한 조치가 적용됨:")
print("  ✅ training_step에서 입력 텐서에 requires_grad 설정")
print("  ✅ ResNet 일부 레이어 freeze 해제")
print("  ✅ TextEmbedder에 gradient 활성화")
print("\n🎯 모델과 Trainer 준비 완료! 다음 셀에서 훈련을 시작하세요.")

In [ ]:
# 모델 훈련 실행
print("🚀 GTM 모델 훈련 시작!")
print("=" * 50)

trainer.fit(
    model, 
    train_dataloaders=train_loader,
    val_dataloaders=test_loader
)

print("\n🎉 훈련 완료!")
print(f"💾 최고 모델: {checkpoint_callback.best_model_path}")

## 6. 📊 결과 분석 및 시각화

In [ ]:
# 훈련 메트릭 시각화
log_dir = './logs/gtm_colab/'
version_dirs = [d for d in os.listdir(log_dir) if d.startswith('version_')]
if version_dirs:
    latest_version = max(version_dirs, key=lambda x: int(x.split('_')[1]))
    metrics_path = os.path.join(log_dir, latest_version, 'metrics.csv')
    
    if os.path.exists(metrics_path):
        metrics_df = pd.read_csv(metrics_path)
        
        # 메트릭 플롯
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss 플롯
        train_loss = metrics_df.dropna(subset=['train_loss'])
        val_loss = metrics_df.dropna(subset=['val_loss'])
        
        axes[0].plot(train_loss['step'], train_loss['train_loss'], label='Training Loss', alpha=0.7)
        axes[0].plot(val_loss['step'], val_loss['val_loss'], label='Validation Loss', marker='o')
        axes[0].set_title('📉 Training/Validation Loss')
        axes[0].set_xlabel('Steps')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # MAE 플롯
        val_mae = metrics_df.dropna(subset=['val_mae'])
        axes[1].plot(val_mae['step'], val_mae['val_mae'], label='Validation MAE', marker='s', color='red')
        axes[1].set_title('📊 Validation MAE')
        axes[1].set_xlabel('Steps')
        axes[1].set_ylabel('MAE')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 최종 성능 출력
        if not val_mae.empty:
            final_mae = val_mae['val_mae'].iloc[-1]
            print(f"🎯 최종 Validation MAE: {final_mae:.2f}")
    else:
        print("⚠️ 메트릭 파일을 찾을 수 없습니다.")

In [ ]:
# 모델 예측 테스트
model.eval()

with torch.no_grad():
    sample_batch = next(iter(test_loader))
    item_sales, category, color, fabric, temporal_features, gtrends_batch, images = sample_batch
    
    # 예측 수행
    predictions, attention_weights = model(category, color, fabric, temporal_features, gtrends_batch, images)
    
    # 정규화 해제
    actual_sales = item_sales * 1065
    predicted_sales = predictions * 1065

# 예측 시각화
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i in range(min(4, len(predictions))):
    actual = actual_sales[i].cpu().numpy()
    predicted = predicted_sales[i].cpu().numpy()
    
    axes[i].plot(months, actual, label='실제 매출', marker='o', linewidth=2)
    axes[i].plot(months, predicted, label='예측 매출', marker='s', linewidth=2, alpha=0.8)
    axes[i].set_title(f'Colab 예측 결과 {i+1}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)
    
    mae = np.mean(np.abs(actual - predicted))
    axes[i].text(0.02, 0.98, f'MAE: {mae:.1f}', transform=axes[i].transAxes, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
                verticalalignment='top')

plt.tight_layout()
plt.show()

# 전체 성능
overall_mae = np.mean(np.abs(actual_sales.cpu().numpy() - predicted_sales.cpu().numpy()))
print(f"🔮 Colab 전체 예측 MAE: {overall_mae:.2f}")

## 7. 💾 모델 저장 (Google Drive)

In [ ]:
# Google Drive에 결과 저장
drive_save_path = '/content/drive/MyDrive/GTM-Results/'
os.makedirs(drive_save_path, exist_ok=True)

# 최고 모델을 Google Drive에 복사
if checkpoint_callback.best_model_path:
    import shutil
    best_model_name = f"gtm_colab_best_{pd.Timestamp.now().strftime('%Y%m%d_%H%M')}.ckpt"
    shutil.copy2(checkpoint_callback.best_model_path, drive_save_path + best_model_name)
    print(f"💾 최고 모델 저장: {drive_save_path + best_model_name}")

# 메트릭 CSV도 저장
if os.path.exists(metrics_path):
    shutil.copy2(metrics_path, drive_save_path + 'training_metrics.csv')
    print(f"📊 훈련 메트릭 저장: {drive_save_path}training_metrics.csv")

# 노트북도 저장
!cp /content/colabtools/notebook.ipynb "{drive_save_path}GTM_Colab_Executed.ipynb"
print(f"📓 실행된 노트북 저장: {drive_save_path}GTM_Colab_Executed.ipynb")

print("\n✅ 모든 결과가 Google Drive에 저장되었습니다!")
print(f"📂 저장 위치: {drive_save_path}")

## 📋 Colab 실행 가이드 (축소된 데이터셋 버전)

### ✅ 실행 전 준비사항

**1. 로컬에서 축소된 데이터셋 생성:**
```bash
# GTM-Transformer 폴더에서 실행
python simple_sample.py    # CSV 파일 1/10로 샘플링  
python copy_images.py      # 필요한 이미지만 복사
```

**2. Google Drive에 업로드할 폴더:**
```
/content/drive/MyDrive/
├── GTM-Transformer/          # 로컬에서 수정된 전체 코드
└── GTM-dataset-small/        # 축소된 데이터셋 (NEW!)
    ├── train.csv            # 508개 샘플 (원본의 1/10)
    ├── test.csv             # 50개 샘플 (원본의 1/10)  
    ├── gtrends.csv          # 동일
    ├── category_labels.pt
    ├── color_labels.pt
    ├── fabric_labels.pt
    └── images/              # 558개 이미지 (원본의 1/10)
        ├── AI17/ (~88개)
        ├── AI18/ (~101개)
        ├── AI19/ (~108개)
        ├── PE17/ (~96개)
        ├── PE18/ (~76개)
        └── PE19/ (~89개)
```

**3. GPU 설정:** 
- Colab에서 런타임 → 런타임 유형 변경 → GPU 선택

### 🚀 실행 순서 (축소된 데이터셋)
1. **패키지 설치** (자동 에러 처리)
2. **Google Drive 마운트 및 코드/데이터셋 확인**
3. **수정된 코드 복사** 
4. **축소된 데이터 로딩 및 빠른 모델 훈련**
5. **결과 분석 및 Google Drive 저장**

### 💡 축소된 데이터셋의 장점
- ✅ **10배 빠른 실행** (~5-10분 내 완료)
- ✅ **GPU 메모리 부족 문제 해결** 
- ✅ **Colab 세션 시간 제한 여유**
- ✅ **빠른 프로토타이핑** 가능
- ✅ **동일한 모델 아키텍처** 테스트

### ⚙️ 수정된 하이퍼파라미터
- **배치 크기**: 8 (GPU) / 4 (CPU)
- **에포크**: 5개 (빠른 실험용)
- **데이터**: 558개 (원본 5,577개의 1/10)
- **예상 훈련 시간**: ~5-10분

### ⚠️ 주의사항
- 축소된 데이터셋이므로 성능은 원본보다 낮을 수 있음
- 실험 및 디버깅 목적으로 최적화됨
- 실제 운영시엔 원본 데이터셋 권장
- 모든 결과는 Google Drive에 자동 저장

### 🔄 원본 데이터셋으로 돌아가려면
경로만 변경: `GTM-dataset-small` → `GTM-dataset`